In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import os
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization

In [ ]:
os.getcwd()

'C:\\Users\\USER\\projects\\lending_club\\notebooks'

In [ ]:
chemin = "../datasets/accepted_2007_to_2018/accepted_2007_to_2018Q4.csv"
data = pd.read_csv(chemin, low_memory = False)

In [ ]:
data.head(4)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 2.5+ GB


In [ ]:
data.shape

(2260701, 151)

In [ ]:
data.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,0.0,2.260668e+06,2.260668e+06,2.260668e+06,2.260668e+06,2.260668e+06,2.260664e+06,2.258957e+06,2.260639e+06,2.260668e+06,...,10917.0,10917.000000,10917.0,10917.000000,8651.000000,10917.000000,10917.000000,34246.000000,34246.000000,34246.000000
mean,NaN,1.504693e+04,1.504166e+04,1.502344e+04,1.309283e+01,4.458068e+02,7.799243e+04,1.882420e+01,3.068792e-01,6.985882e+02,...,3.0,155.045981,3.0,13.743886,454.798089,11636.883942,193.994321,5010.664267,47.780365,13.191322
std,NaN,9.190245e+03,9.188413e+03,9.192332e+03,4.832138e+00,2.671735e+02,1.126962e+05,1.418333e+01,8.672303e-01,3.301038e+01,...,0.0,129.040594,0.0,9.671178,375.385500,7625.988281,198.629496,3693.122590,7.311822,8.159980
min,NaN,5.000000e+02,5.000000e+02,0.000000e+00,5.310000e+00,4.930000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,6.100000e+02,...,3.0,0.640000,3.0,0.000000,1.920000,55.730000,0.010000,44.210000,0.200000,0.000000
25%,NaN,8.000000e+03,8.000000e+03,8.000000e+03,9.490000e+00,2.516500e+02,4.600000e+04,1.189000e+01,0.000000e+00,6.750000e+02,...,3.0,59.440000,3.0,5.000000,175.230000,5627.000000,44.440000,2208.000000,45.000000,6.000000
50%,NaN,1.290000e+04,1.287500e+04,1.280000e+04,1.262000e+01,3.779900e+02,6.500000e+04,1.784000e+01,0.000000e+00,6.900000e+02,...,3.0,119.140000,3.0,15.000000,352.770000,10028.390000,133.160000,4146.110000,45.000000,14.000000
75%,NaN,2.000000e+04,2.000000e+04,2.000000e+04,1.599000e+01,5.933200e+02,9.300000e+04,2.449000e+01,0.000000e+00,7.150000e+02,...,3.0,213.260000,3.0,22.000000,620.175000,16151.890000,284.190000,6850.172500,50.000000,18.000000
max,NaN,4.000000e+04,4.000000e+04,4.000000e+04,3.099000e+01,1.719830e+03,1.100000e+08,9.990000e+02,5.800000e+01,8.450000e+02,...,3.0,943.940000,3.0,37.000000,2680.890000,40306.410000,1407.860000,33601.000000,521.350000,181.000000


In [ ]:
target = "loan_status"

In [ ]:
data = data[(data[target] == "Fully Paid") | (data[target] == "Charged Off")]

In [ ]:
not_defined = (data.isna().sum().sort_values(ascending = False) / data.shape[0]) * 100

In [ ]:
eliminated = not_defined[not_defined > 60]
eliminated.shape

(57,)

In [ ]:
index = eliminated.index
data = data.drop(index, axis = 1)
data.shape

(1345310, 94)

In [ ]:
(data.isna().sum() / data.shape[0]).sort_values(ascending = False)

mths_since_last_delinq    0.504525
mths_since_recent_inq     0.129391
num_tl_120dpd_2m          0.087267
mo_sin_old_il_acct        0.078476
emp_title                 0.063766
                            ...   
initial_list_status       0.000000
total_acc                 0.000000
revol_bal                 0.000000
pub_rec                   0.000000
debt_settlement_flag      0.000000
Length: 94, dtype: float64

In [ ]:
#
numeric_variable = []
categorical_variable = []
for elt in data.select_dtypes(["float", "int"]):
    numeric_variable.append(elt)

for elt in data.select_dtypes("object"):
    categorical_variable.append(elt)

In [ ]:
len(categorical_variable) + len(numeric_variable) == data.shape[1]

True

In [ ]:
#suppression de "loan status" dans les variables categorielles
i = 0
for elt in categorical_variable:
    if (elt == "loan_status") | (elt == "id"):
        del categorical_variable[i]

    i +=1

In [ ]:
dictionnaire = {
    'Fully Paid' : "1",
    'Charged Off':"0"
}
data["loan_status"] = data["loan_status"].replace(dictionnaire).astype("int")

In [ ]:
##pipeline de preprocessing pour les variables numeriques

numeric_pipe = Pipeline(steps = [
    ("Imputation Numerique", SimpleImputer(strategy = "mean")),
    ("Normalisation", StandardScaler())
])


In [ ]:
#pipeline de preprocessing pour les variables categorielles
categorical_pipe = Pipeline(steps = [
    ("Imputation categorielle", SimpleImputer(strategy = "most_frequent")),
    ("Encodage des variables", OneHotEncoder(handle_unknown="ignore")),
])

In [ ]:
preprocessor = ColumnTransformer(transformers = [
    ("numerique", numeric_pipe, numeric_variable),
    ("categorielle", categorical_pipe, categorical_variable)
])

In [ ]:
#pipeline de preprocessing

preprocessed_pipeline = Pipeline(steps = [
    ("preprocessing", preprocessor)
])

In [ ]:
# division de la base de donnees en entrainement, validation et test
X = data.drop(["loan_status", "id"], axis = 1)
Y = data["loan_status"]

Xtrain, Xval, Ytrain, Yval = train_test_split(X, Y, test_size = 0.4, random_state = 42)
Xval, Xtest, Yval, Ytest = train_test_split(Xval, Yval, test_size = 0.3, random_state = 42)

In [ ]:
Xtrain.shape

(807186, 92)

In [ ]:
Xval.shape

(376686, 92)

In [ ]:
Xtest.shape

(161438, 92)

In [ ]:
data["loan_status"].unique()

array([1, 0])

In [ ]:
Xtrain_preprocessed = preprocessor.fit_transform(Xtrain).astype('float')
Xtest_preprocessed = preprocessor.transform(Xtest).astype('float')

In [ ]:
Xtrain_preprocessed.shape

(807186, 1100171)

# Creation du reseau de neuronnes

In [ ]:
# Creation du modele

model = Sequential()
model.add(Input((Xtrain_preprocessed.shape[1],)))
model.add(Dense(units = 524 , activation = "relu"))
model.add(Dense(units = 256))
model.add(BatchNormalization())
model.add(Dense(units = 128, activation = "relu"))
model.add(Dense(units = 1, activation = "sigmoid"))


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 524)                 │     576,490,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         134,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 576,658,577 (2.15 GB)

 Trainable params: 576,658,065 (2.15 GB)

 Non-trainable params: 512 (2.00 KB)

In [ ]:
model.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)

In [ ]:
history = model.fit(
    Xtrain_preprocessed, Ytrain,
    validation_data = (Xtest_preprocessed, Ytest),
    epochs = 2,
    verbose = True
)

Epoch 1/2
  124/25225 ━━━━━━━━━━━━━━━━━━━━ 47:23:56 7s/step - accuracy: 0.9259 - loss: 0.2051

In [1]:
# La Ram ne supporte pas l'execution de l'entrainement